# Import des bibliothèques

In [610]:
import numpy as np
from scipy import optimize
import pandas as pd
from scipy.optimize import root
import warnings
warnings.filterwarnings("ignore")

# Définition des trois modèles

## 2SFCA :

$$
W_{i j}=\frac{1}{d_{i j}} \hspace{5mm} \text{et} \hspace{5mm} \mathbb{P}_{i j} \propto W_{i j} \hspace{2.5mm} \text{ i.e } \hspace{2.5mm} \mathbb{P}_{i j}=\frac{W_{i j}}{\sum_{k} W_{i k}}
$$

In [611]:
def Prob_eval_SFCA2(d):
    """
    Calcule la probabilité qu'un patient d'une commune i visite un médecin d'une commune j
    en utilisant le modèle SFCA2.

    Args:
        d (numpy.array): Une matrice de distances. L'élément [i, j] représente la distance
            entre la commune i et la commune j.

    Returns:
        Prob (numpy.array): Une matrice de probabilités de même dimension que d. L'élément [i, j] 
            représente la probabilité qu'un patient de la commune i visite un médecin de la commune j.
    """
    W = 1.0 / d
    Prob = W / np.sum(W, axis=1, keepdims=True)
    
    return Prob



## 3SFCA :

$$
W_{i j}=\frac{1}{d_{i j}} \hspace{5mm} \text { et } \hspace{5mm} \mathbb{P}_{i j} \propto W_{i j} S_{j} \hspace{2.5mm} \text { i.e } \hspace{2.5mm} \mathbb{P}_{i j}=\frac{W_{i j} S_{j}}{\sum_{k} W_{i k} S_{k}}
$$


In [612]:
def Prob_eval_SFCA3(d, S):
    """
    Calcule la matrice de probabilité qu'un patient d'une commune i visite un médecin 
    d'une commune j en utilisant le modèle SFCA3.
    La fonction prend comme entrée une matrice de distances, un vecteur de demandes d'offres 
    par commune, et un vecteur d'offres de soins par commune. Elle retourne une matrice 
    de probabilités correspondante.

    Args:
        d (numpy.array): Une matrice de distances. L'élément [i, j] représente 
            la distance entre la commune i et la commune j.
        S (numpy.array): Un vecteur de l'offre de soins pour chaque commune j.

    Returns:
        Prob (numpy.array): Une matrice de probabilités de même dimension que d. L'élément [i, j] 
            représente la probabilité qu'un patient de la commune i visite un médecin de la commune j.
    """
    W = 1.0 / d
    WS = W * S
    Prob = WS / np.sum(WS, axis=1, keepdims=True)
    return Prob

## Point Fixe :
$$
W_{i j}=\frac{1}{d_{i j}} \hspace{5mm} \text { et } \hspace{5mm} \mathbb{P}_{i j} \propto W_{i j} R_{j} \hspace{2.5mm} \text { i.e } \hspace{2.5mm} \mathbb{P}_{i j}=\frac{W_{i j} R_{j}}{\sum_{k} W_{i k} R_{k}}
$$

### Définition des fonctions du point fixe

$$
F: \left\{ 
\begin{array}{cl}
\mathbb{R}^{J} \times \mathbb{R}^{N \times J} & \rightarrow \mathbb{R}^{J} \times \mathbb{R}^{N \times J} \\
\left( R=\left(R_{j}\right)_{j}, \mathbb{P}=\left(\mathbb{P}_{i j}\right)_{i j} \right) & \mapsto \left( F_{1}(\mathbb{P}), F_{2}(R) \right) \\
\end{array} 
\right.
$$

où $ F_{1}(\mathbb{P})=\left(\frac{S_{j}}{\sum_{i} P_{i} \mathbb{P}_{i j}}\right)_{j} $ et  $ F_{2}(R)=\left(\frac{R_{j} W_{i j}}{\sum_{k} R_{k} W_{i k}}\right)_{i j} $


In [613]:
def calc_Rj(P, Prob, S):
    """
    Calcule le vecteur R où chaque élément R[j] est donné par la formule :
    R_j = S_j / (sum_i P_i * Prob_{i,j}).


    Args:
        P (numpy.array): Un vecteur de la demande d'offre pour chaque commune.
        Prob (numpy.array): Une matrice de probabilités. L'élément [i, j] représente 
            la probabilité qu'un patient de la commune i visite un médecin de la commune j.
        S (numpy.array): Un vecteur de l'offre de soins pour chaque commune.

    Returns:
        numpy.array: Un vecteur R. Chaque élément R[j] est donné par la formule : 
            R_j = S_j / (sum_i P_i * Prob_{i,j}).
    """
    sum_prob_p = np.sum(P[:, np.newaxis] * Prob, axis=0)
    sum_prob_p [sum_prob_p==0]+= 1e-2
    R = S / sum_prob_p
    return R


def calc_Prob_ij(R, W):
    """
    Calcule la matrice de probabilités Prob où chaque élément Prob[i, j] selon le modele du point fixe donné par la formule :
    Prob[i, j] = R[j] * W[i, j] / (sum_k R[k] * W[i, k]).

    Args:
        R (numpy.array): Un vecteur des offres totales par patients potentiels pour chaque commune.
        W (numpy.array): Une matrice de perméabilité. L'élément [i, j] représente la perméabilité 
            entre la commune i et la commune j.

    Returns:
        numpy.array: Une matrice de probabilités de même dimension que W. L'élément [i, j] 
            représente la probabilité.
    """
    RW = R * W
    Prob = RW / np.sum(RW, axis=1, keepdims=True)
    return Prob

def F(R, Prob, W, P, S):
    """
    Calcule le nouveau R et Prob en utilisant les fonctions calc_Rj et calc_Prob_ij.

    Args:
        R (numpy.array): Un vecteur des offres totales par patients potentiels pour chaque commune.
        Prob (numpy.array): Une matrice de probabilités.
        W (numpy.array): Une matrice des coefficients de perméabilité.
        P (numpy.array): Un vecteur des demandes (nombre de patients dans la région i).
        S (numpy.array): Un vecteur des offres (par exemple, le nombre d'heures de travail des médecins dans la région j).

    Returns:
        R_new (numpy.array): Le nouveau vecteur R calculé par la fonction calc_Rj.
        Prob_new (numpy.array): La nouvelle matrice de probabilités calculée par la fonction calc_Prob_ij.
    """
    Prob_new = calc_Prob_ij(R, W)
    R_new = calc_Rj(P, Prob, S)
    return R_new, Prob_new


### Méthode 1 :
algorithme classique

In [614]:
def Point_fixe_SFCA(W, S, P, maxiter = 10000):
    """
    Implémentation de l'algorithme pour calculer le vecteur R et la matrice Prob avec la méthode du point fixe.

    Args:
        W (numpy.array): Une matrice des coefficients de perméabilité.
        S (numpy.array): Un vecteur des offres (par exemple, le nombre d'heures de travail des médecins dans la région j).
        P (numpy.array): Un vecteur des demandes (nombre de patients dans la région i).
        maxiter (int): Le nombre l'itération maximale.

    Returns:
        R (numpy.array): Un vecteur des offres totales par patients potentiels.
        Prob (numpy.array): Une matrice de probabilité de connexions.
    """

    R = np.ones_like(S)

    for _ in range(maxiter):
        
        P_prime = R * W
        P_prime_row_sum = np.sum(P_prime, axis=1, keepdims=True)
        Prob = P_prime / P_prime_row_sum

        R = S / np.dot(P.T, Prob)

    return R, Prob

In [615]:
def find_fixed_point(F, W, P, S, max_iter=10000):
    """
    Trouve le point fixe de la fonction F.

    Args:
        F (function): La fonction pour laquelle trouver le point fixe.
        W (numpy.array): Une matrice des coefficients de perméabilité.
        P (numpy.array): Un vecteur des demandes (nombre de patients dans la région i).
        S (numpy.array): Un vecteur des offres (par exemple, le nombre d'heures de travail des médecins dans la région j).
        tol (float): La tolérance pour la convergence.
        max_iter (int): Le nombre maximal d'itérations.

    Returns:
        R (numpy.array): Le vecteur R au point fixe.
        Prob (numpy.array): La matrice de probabilités au point fixe.
    """

    R = np.ones_like(S)
    Prob = np.random.rand(len(P), len(S))

    for _ in range(max_iter) :
        R_new, Prob_new = F(R, Prob, W, P, S)
        R = R_new
        Prob = Prob_new


    return R, Prob


### Méthode 2 : 
En utilisant Scipy

In [616]:
def F_diff(X, W, P, S):
    R, Prob = X[:len(S)], X[len(S):].reshape(W.shape)
    R_new, Prob_new = F(R, Prob, W, P, S)
    return np.concatenate([R_new - R, (Prob_new - Prob).ravel()])

def find_fixed_point_2(W, P, S, tol=1e-10, max_iter=10000):
    R_init = np.ones_like(S)
    Prob_init = np.random.rand(len(P), len(S))
    X_init = np.concatenate([R_init, Prob_init.ravel()])
    sol = root(F_diff, X_init, args=(W, P, S))
    if sol.success:
        X_fixed = sol.x
        R_fixed, Prob_fixed = X_fixed[:len(S)], X_fixed[len(S):].reshape(W.shape)
        return R_fixed, Prob_fixed
    else:
        print("La méthode n'a pas convergé.")



## Fonction de décision 
Si $\hspace{3mm} FCA_i = \sum_j  \mathbb{P}_{i j} R_j \hspace{1mm} \leq 1/10  \hspace{3mm}$ alors $i$ est un désert médical

In [617]:
def deserts_medicaux(d, S, P, model="SFCA3", seuil  = 0.1):
    """
    Détermine si chaque commune est un désert médical ou non en utilisant l'un des trois modèles.

    Args:
        d (numpy.array): Matrice de distances entre les communes.
        S (numpy.array): Vecteur des offres de soins pour chaque commune.
        P (numpy.array): Vecteur des demandes de chaque commune.
        model (str): Modèle à utiliser pour calculer les probabilités. Par défaut, "SFCA3".
        Seuil = Float, le sueil de décision

    Returns:
        numpy.array: Vecteur binaire indiquant si chaque commune est un désert médical ou non.
    """
    R_calcule = False
    # Calcul des probabilités selon le modèle spécifié
    if model == "SFCA2":
        Prob = Prob_eval_SFCA2(d)
    elif model == "SFCA3":
        Prob = Prob_eval_SFCA3(d, S)
    elif model =="point fixe" :
        R_calcule = True
        R, Prob = Point_fixe_SFCA(1.0/d, S, P, maxiter= 1000)
        #R, Prob = find_fixed_point_2(1.0/d, P, S)
    else:
        raise ValueError("Modèle non valide. Veuillez choisir parmi 'point fixe', 'SFCA2', ou 'SFCA3'.")

    if R_calcule == False :
        R = calc_Rj(P, Prob, S)
    # Calcul de la somme Pij * Rj pour chaque commune i
    print(R)
    sum_PR = np.sum(Prob * R, axis=1)

    # Vérification si chaque commune est un désert médical ou non
    FCA = sum_PR <= seuil

    return FCA


# Import des données et prétraitement 

In [618]:
#! pip install openpyxl
distancier = pd.read_csv("distancier.csv", sep=";")
population = pd.read_excel("Medecins.xlsx")
medecins = pd.read_excel("Population.xlsx")

### Formater et nettoyer le distancier
Dans cet exemple on a pris les communes de la region Centre val de Loire

In [619]:
distancier.head(10)

,ID,idSrc,libSrc,lonSrc,latSrc,idDst,libDst,lonDst,latDst,distance
0,1,18001,Achères,"2,4478","47,2778",18001,Achères,"2,4478","47,2778",0
1,2,18001,Achères,"2,4478","47,2778",18002,Ainay-le-Vieil,"2,55292","46,66382","95,294"
2,3,18001,Achères,"2,4478","47,2778",18003,Les Aix-d'Angillon,"2,57138","47,19794","19,381"
3,4,18001,Achères,"2,4478","47,2778",18004,Allogny,"2,31974","47,22555","11,968"
4,5,18001,Achères,"2,4478","47,2778",18005,Allouis,"2,2279","47,16602","22,126"
5,6,18001,Achères,"2,4478","47,2778",18006,Annoix,"2,53287","46,95995","41,84"
6,7,18001,Achères,"2,4478","47,2778",18007,Apremont-sur-Allier,"3,04761","46,90648","80,044"
7,8,18001,Achères,"2,4478","47,2778",18008,Arçay,"2,34174","46,94903","43,586"
8,9,18001,Achères,"2,4478","47,2778",18009,Arcomps,"2,43275","46,67542","90,729"
9,10,18001,Achères,"2,4478","47,2778",18010,Ardenais,"2,35773","46,64708","98,25"


In [620]:
distancier = distancier[["idSrc", "idDst", "distance"]]
print(distancier.dtypes)
distancier

idSrc        int64
idDst        int64
distance    object
dtype: object


,idSrc,idDst,distance
0,18001,18001,0
1,18001,18002,"95,294"
2,18001,18003,"19,381"
3,18001,18004,"11,968"
4,18001,18005,"22,126"
...,...,...,...
3087044,45348,45344,"85,873"
3087045,45348,45345,"42,839"
3087046,45348,45346,"30,2"
3087047,45348,45347,"14,56"


In [621]:
distancier["distance"] = distancier["distance"].str.replace(',', '.').astype(float)
print( (distancier['distance'] < 0).sum() )
#remplacer les valeur nulles et négatives pour eviter certains problemes
distancier.loc[distancier['distance'] == 0, 'distance'] = 1
distancier.loc[distancier['distance'] < 0, 'distance'] = -distancier['distance']
distancier

0


,idSrc,idDst,distance
0,18001,18001,1.000
1,18001,18002,95.294
2,18001,18003,19.381
3,18001,18004,11.968
4,18001,18005,22.126
...,...,...,...
3087044,45348,45344,85.873
3087045,45348,45345,42.839
3087046,45348,45346,30.200
3087047,45348,45347,14.560


### Formater et nettoyer les données dur la population

In [622]:
population.head(10)

,Chiffres détaillés - Séries historiques de population (1876 à 2020),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,France hors Mayotte - Communes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mise en ligne : décembre 2022 Géographie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"©Insee Source(s) : Insee, Recensements d...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Code géographique,Région,Département,Libellé géographique,Population en 2020,Population en 2019,Population en 2018,Population en 2017,Population en 2016,Population en 2015,...,Population en 1926,Population en 1921,Population en 1911,Population en 1906,Population en 1901,Population en 1896,Population en 1891,Population en 1886,Population en 1881,Population en 1876
4,CODGEO,REG,DEP,LIBGEO,PMUN20,PMUN19,PMUN18,PMUN17,PMUN16,PMUN15,...,PTOT1926,PTOT1921,PTOT1911,PTOT1906,PTOT1901,PTOT1896,PTOT1891,PTOT1886,PTOT1881,PTOT1876
5,01001,84,01,L' Abergement-Clémenciat,806,779,771,776,767,767,...,543,566,560,629,637,572,594,622,605,604
6,01002,84,01,L' Abergement-de-Varey,262,256,253,248,243,241,...,253,256,291,323,357,391,421,486,490,484
7,01004,84,01,Ambérieu-en-Bugey,14288,14134,14204,14035,14081,14127,...,5705,4796,4334,4100,4023,3548,3635,3618,3396,3427
8,01005,84,01,Ambérieux-en-Dombes,1782,1751,1720,1689,1671,1619,...,690,698,805,805,835,852,862,882,879,848
9,01006,84,01,Ambléon,113,112,112,111,110,109,...,161,150,159,193,200,206,200,200,213,191


In [623]:
population = population.iloc[5:]
population = population[population.columns[:5]]
population.columns = ["codego", "REG", "DEP", "libgeo", "population"]
population['codego'] = pd.to_numeric(population['codego'], errors='coerce')
population = population.dropna(subset=['codego'])
population["population"] = population["population"].astype(float)
population.head(10)

,codego,REG,DEP,libgeo,population
5,1001.0,84,01,L' Abergement-Clémenciat,806.0
6,1002.0,84,01,L' Abergement-de-Varey,262.0
7,1004.0,84,01,Ambérieu-en-Bugey,14288.0
8,1005.0,84,01,Ambérieux-en-Dombes,1782.0
9,1006.0,84,01,Ambléon,113.0
10,1007.0,84,01,Ambronay,2827.0
11,1008.0,84,01,Ambutrix,768.0
12,1009.0,84,01,Andert-et-Condon,324.0
13,1010.0,84,01,Anglefort,1101.0
14,1011.0,84,01,Apremont,368.0


### Formater et nettoyer les données sur les medecins
Ici on teste que sur les medecins generalistes

In [624]:
medecins.head(10)

,Observatoire des territoires - ANCT,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Offre de services de santé (Commune (2021)),NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Nombre de médecins généralistes libéraux
3,codgeo,libgeo,an,nb_medg
4,01001,L'Abergement-Clémenciat,2020,0
5,01002,L'Abergement-de-Varey,2020,0
6,01004,Ambérieu-en-Bugey,2020,24
7,01005,Ambérieux-en-Dombes,2020,2
8,01006,Ambléon,2020,0
9,01007,Ambronay,2020,3


In [625]:
medecins = medecins.iloc[4:]
medecins.columns = ["codego", "libgeo", "annee", "medecins"]
medecins['codego'] = pd.to_numeric(medecins['codego'], errors='coerce')
medecins = medecins.dropna(subset=['codego'])
medecins["medecins"] = medecins["medecins"].astype(float)
medecins

,codego,libgeo,annee,medecins
4,1001.0,L'Abergement-Clémenciat,2020,0.0
5,1002.0,L'Abergement-de-Varey,2020,0.0
6,1004.0,Ambérieu-en-Bugey,2020,24.0
7,1005.0,Ambérieux-en-Dombes,2020,2.0
8,1006.0,Ambléon,2020,0.0
...,...,...,...,...
34964,97613.0,M'Tsangamouji,2020,0.0
34965,97614.0,Ouangani,2020,0.0
34966,97615.0,Pamandzi,2020,3.0
34967,97616.0,Sada,2020,3.0


In [626]:
# Jointure entre df_medecins et df_population
df_merge1 = medecins.merge(population, on='codego', how='inner')

# Jointure entre df_merge1 et df_distancier
df_final = df_merge1.merge(distancier, left_on='codego', right_on='idSrc', how='inner')

df_final

,codego,libgeo_x,annee,medecins,REG,DEP,libgeo_y,population,idSrc,idDst,distance
0,18001.0,Achères,2020,0.0,24,18,Achères,364.0,18001,18001,1.000
1,18001.0,Achères,2020,0.0,24,18,Achères,364.0,18001,18002,95.294
2,18001.0,Achères,2020,0.0,24,18,Achères,364.0,18001,18003,19.381
3,18001.0,Achères,2020,0.0,24,18,Achères,364.0,18001,18004,11.968
4,18001.0,Achères,2020,0.0,24,18,Achères,364.0,18001,18005,22.126
...,...,...,...,...,...,...,...,...,...,...,...
3087044,45348.0,Yèvre-la-Ville,2020,0.0,24,45,Yèvre-la-Ville,694.0,45348,45344,85.873
3087045,45348.0,Yèvre-la-Ville,2020,0.0,24,45,Yèvre-la-Ville,694.0,45348,45345,42.839
3087046,45348.0,Yèvre-la-Ville,2020,0.0,24,45,Yèvre-la-Ville,694.0,45348,45346,30.200
3087047,45348.0,Yèvre-la-Ville,2020,0.0,24,45,Yèvre-la-Ville,694.0,45348,45347,14.560


### Créer notre matrice de distance 
De plus on remarque que cela est possible car la taille de notre dataframe 3087049 est un carré

In [627]:
d_df = df_final.pivot(index='idSrc', columns='idDst', values='distance')
d_df

idDst,18001,18002,18003,18004,18005,18006,18007,18008,18009,18010,...,45339,45340,45341,45342,45343,45344,45345,45346,45347,45348
idSrc,,,,,,,,,,,,,,,,,,,,,
18001,1.000,95.294,19.381,11.968,22.126,41.840,80.044,43.586,90.729,98.250,...,104.163,52.066,132.574,133.325,103.345,128.004,101.757,88.680,102.320,129.364
18002,95.338,1.000,72.694,83.509,78.488,39.916,52.751,42.430,14.642,16.849,...,216.036,161.957,200.189,200.940,215.218,195.619,213.630,184.102,224.867,228.348
18003,19.414,72.844,1.000,22.059,39.274,34.994,59.714,36.740,82.899,90.420,...,125.154,63.235,172.894,173.644,124.335,168.323,122.748,99.848,170.980,150.355
18004,11.965,83.463,22.071,1.000,10.296,42.397,80.601,38.735,78.898,86.419,...,112.376,60.279,124.072,124.823,111.558,119.502,109.970,88.337,148.750,152.231
18005,22.124,78.030,39.494,10.296,1.000,38.365,82.075,33.302,73.465,80.986,...,122.535,70.438,128.404,129.155,121.716,123.834,120.128,112.317,153.082,156.563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45344,126.817,193.362,166.816,118.240,122.530,160.989,205.681,150.134,188.797,196.317,...,106.389,68.670,36.993,58.067,108.765,1.000,124.145,71.924,81.994,85.475
45345,101.808,213.310,122.853,109.861,120.020,157.807,155.833,159.554,205.713,213.233,...,17.348,46.571,106.217,73.171,17.103,124.690,1.000,41.628,50.149,42.549
45346,88.878,183.187,99.981,88.430,112.355,150.814,183.249,139.959,178.622,186.143,...,27.435,37.371,63.855,35.291,32.852,71.856,41.768,1.000,19.574,30.442


In [628]:
d = d_df.values

### Extraire nos vecteurs $ \hspace{2mm} P = (P_i)_i \hspace{2mm} $ et $ \hspace{2mm} S = (S_j)_j \hspace{2mm} $

In [629]:
S_P = df_final[["codego","medecins", "population"]].drop_duplicates()
S_P

,codego,medecins,population
0,18001.0,0.0,364.0
1757,18002.0,0.0,191.0
3514,18003.0,1.0,1881.0
5271,18004.0,0.0,1097.0
7028,18005.0,0.0,1074.0
...,...,...,...
3078264,45344.0,0.0,1084.0
3080021,45345.0,1.0,1118.0
3081778,45346.0,1.0,2269.0
3083535,45347.0,0.0,811.0


In [630]:
S = S_P["medecins"].values
P = S_P["population"].values

# Tester nos modèles sur ces données
afficher le pourcentage de deserts medicaux dans cette region

In [632]:
modeles = ["SFCA2", "SFCA3", "point fixe"]
resultats = {}

for modele in modeles:
    print(f"Modèle : {modele}")
    is_desert = deserts_medicaux(d, S, P, model=modele)
    nb_deserts = np.sum(is_desert)
    pourcentage_deserts = (nb_deserts / len(is_desert)) * 100

    print(f"Nombre de déserts médicaux : {nb_deserts}")
    print(f"Pourcentage de déserts médicaux : {pourcentage_deserts:.2f}%")

    resultats[modele] = {
        "is_desert": is_desert,
        "nb_deserts": nb_deserts,
        "pourcentage_deserts": pourcentage_deserts
    }

    print("----------------------------------------------------")



Modèle : SFCA2
[0.         0.         0.00075091 ... 0.00060297 0.         0.        ]
Nombre de déserts médicaux : 1757
Pourcentage de déserts médicaux : 100.00%
----------------------------------------------------
Modèle : SFCA3
[0.         0.         0.00105369 ... 0.00090457 0.         0.        ]
Nombre de déserts médicaux : 1757
Pourcentage de déserts médicaux : 100.00%
----------------------------------------------------
Modèle : point fixe
[nan nan nan ... nan nan nan]
Nombre de déserts médicaux : 0
Pourcentage de déserts médicaux : 0.00%
----------------------------------------------------


In [633]:
for modele, resultat in resultats.items():
    print(f"Modèle : {modele}")
    print(f"Nombre de déserts médicaux : {resultat['nb_deserts']}")
    print(f"Pourcentage de déserts médicaux : {resultat['pourcentage_deserts']:.2f}%")
    print("----------------------------------------------------")

Modèle : SFCA2
Nombre de déserts médicaux : 1757
Pourcentage de déserts médicaux : 100.00%
----------------------------------------------------
Modèle : SFCA3
Nombre de déserts médicaux : 1757
Pourcentage de déserts médicaux : 100.00%
----------------------------------------------------
Modèle : point fixe
Nombre de déserts médicaux : 0
Pourcentage de déserts médicaux : 0.00%
----------------------------------------------------
